## Bayesian Networks

Now we’ll play with a tiny Bayesian Network (BN) — **Rain → Sprinkler → WetGrass** — and do **belief updates** by hand, using simple Python.
No external packages needed. Change the numbers, re-run the cells, watch beliefs shift. 

**Network:**

```
Rain ─▶ Sprinkler ─▶ WetGrass
   └──────────────────────▶
```

- `Rain ∈ {0,1}`
- `Sprinkler ∈ {0,1}`
- `WetGrass ∈ {0,1}`

We’ll use **tables** (priors, likelihoods) and **normalization** — just like the previous notebook — but for *multiple variables*.



## Reference diagram (fixed Conditional Probability Table (CPT)s)

The following image taken from chegg.com matches a common **Rain–Sprinkler–GrassWet** example.  
We'll also provide a code cell to **set the CPTs to these exact numbers** so the notebook's calculations align with the figure.

![BN reference](images/chegg_BN.png)


In [3]:

# Set CPTs to match the reference image

cpts = {
    'Rain': {0: 0.8, 1: 0.2},  # P(R=F)=0.8, P(R=T)=0.2
    # Sprinkler | Rain  (S, R): probabilities for S state given R
    # From table under "SPRINKLER": when R=F -> [T:0.4, F:0.6]; when R=T -> [T:0.01, F:0.99]
    'Sprinkler': {
        #(S,R): P(S|R)
        (0,0): 0.6,  # P(S=0|R=0)
        (1,0): 0.4,  # P(S=1|R=0)
        (0,1): 0.99, # P(S=0|R=1)
        (1,1): 0.01, # P(S=1|R=1)
    },
    # GrassWet | Sprinkler, Rain  (W, S, R)
    # Table (columns T,F for W; rows by S,R in order (F,F), (F,T), (T,F), (T,T)):
    # (F,F): T 0.0, F 1.0
    # (F,T): T 0.8, F 0.2
    # (T,F): T 0.9, F 0.1
    # (T,T): T 0.99, F 0.01
    'WetGrass': {
        #(W,S,R):
        (1,0,0): 0.0,  (0,0,0): 1.0,
        (1,0,1): 0.8,  (0,0,1): 0.2,
        (1,1,0): 0.9,  (0,1,0): 0.1,
        (1,1,1): 0.99, (0,1,1): 0.01,
    }
}


## Exact inference by enumeration
We'll use tiny tables and normalization.


In [4]:

from itertools import product

def normalize(pdict):
    z = sum(pdict.values())
    if z == 0:
        raise ValueError("All probabilities are zero; check your CPTs/evidence.")
    return {k: v / z for k, v in pdict.items()}

def all_assignments(vars_list):
    for bits in product([0,1], repeat=len(vars_list)):
        yield dict(zip(vars_list, bits))

def P_joint(assign, cpts):
    r = assign['Rain']
    s = assign['Sprinkler']
    w = assign['WetGrass']
    p = 1.0
    p *= cpts['Rain'][r]
    p *= cpts['Sprinkler'][(s, r)]
    p *= cpts['WetGrass'][(w, s, r)]
    return p

def query_posterior(query_var, evidence, cpts):
    vars_all = ['Rain','Sprinkler','WetGrass']
    hidden = [v for v in vars_all if v != query_var and v not in evidence]
    post_unnorm = {}
    for q_val in [0,1]:
        total = 0.0
        for h_assign in all_assignments(hidden):
            assign = {query_var: q_val, **evidence, **h_assign}
            total += P_joint(assign, cpts)
        post_unnorm[q_val] = total
    return normalize(post_unnorm)



## Define the BN (change these numbers!)

- `P(Rain)`  
- `P(Sprinkler | Rain)`  
- `P(WetGrass | Sprinkler, Rain)`  


In [5]:

# # --- CPTs (you can uncomment and edit these to play if you wish!) ---
# cpts = {
#     # Prior for Rain
#     'Rain': {0: 0.7, 1: 0.3},
#     # Sprinkler depends on Rain  (S, R)
#     'Sprinkler': {
#         (0,0): 0.2, (1,0): 0.8,
#         (0,1): 0.7, (1,1): 0.3,
#     },
#     # WetGrass depends on Sprinkler and Rain  (W, S, R)
#     'WetGrass': {
#         (1,1,1): 0.99, (0,1,1): 0.01,
#         (1,1,0): 0.90, (0,1,0): 0.10,
#         (1,0,1): 0.80, (0,0,1): 0.20,
#         (1,0,0): 0.05, (0,0,0): 0.95,
#     }
# }

def check_cpts(cpts):
    assert abs(cpts['Rain'][0] + cpts['Rain'][1] - 1.0) < 1e-9
    for r in [0,1]:
        s0 = cpts['Sprinkler'][(0,r)]
        s1 = cpts['Sprinkler'][(1,r)]
        assert abs(s0 + s1 - 1.0) < 1e-9
    for s in [0,1]:
        for r in [0,1]:
            w1 = cpts['WetGrass'][(1,s,r)]
            w0 = cpts['WetGrass'][(0,s,r)]
            assert abs(w0 + w1 - 1.0) < 1e-9
    return "CPTs look valid."

check_cpts(cpts)


'CPTs look valid.'


## Queries


In [6]:

# P(WetGrass=1)
p_wet = 0.0
for r in [0,1]:
    for s in [0,1]:
        p_wet += cpts['Rain'][r] * cpts['Sprinkler'][(s,r)] * cpts['WetGrass'][(1,s,r)]
print(f"P(WetGrass=1) = {p_wet:.3f}")


P(WetGrass=1) = 0.448


In [5]:

print("P(Rain | WetGrass=1) =", {k: round(v,3) for k,v in query_posterior('Rain', {'WetGrass':1}, cpts).items()})


P(Rain | WetGrass=1) = {0: 0.642, 1: 0.358}


In [6]:
print("P(Sprinkler | WetGrass=1) =", {k: round(v,3) for k,v in query_posterior('Sprinkler', {'WetGrass':1}, cpts).items()})

P(Sprinkler | WetGrass=1) = {0: 0.353, 1: 0.647}


In [7]:
print("P(Rain | WetGrass=1, Sprinkler=0) =", {k: round(v,3) for k,v in query_posterior('Rain', {'WetGrass':1,'Sprinkler':0}, cpts).items()})

P(Rain | WetGrass=1, Sprinkler=0) = {0: 0.0, 1: 1.0}



## Mini-Exercises

1. **Make rain more common.** Set `P(Rain=1)` to `0.6`. What happens to `P(Rain | WetGrass=1)`?
2. **Sprinkler suppression.** Set `P(S=1|R=1)=0.05`. How do the posteriors change?
3. **Noisy grass sensor.** Increase `P(W=1|S=0,R=0)` to `0.15`. Does wet grass still indicate rain?
4. **Your own story.** Rename variables (e.g., `Cold → Flu → Cough`) and pick plausible CPTs.


In [9]:
cpts = {
    'Rain': {0: 0.4, 1: 0.6},  # P(R=F)=0.8, P(R=T)=0.2
    # Sprinkler | Rain  (S, R): probabilities for S state given R
    # From table under "SPRINKLER": when R=F -> [T:0.4, F:0.6]; when R=T -> [T:0.01, F:0.99]
    'Sprinkler': {
        #(S,R): P(S|R)
        (0,0): 0.6,  # P(S=0|R=0)
        (1,0): 0.4,  # P(S=1|R=0)
        (0,1): 0.99, # P(S=0|R=1)
        (1,1): 0.01, # P(S=1|R=1)
    },
    # GrassWet | Sprinkler, Rain  (W, S, R)
    # Table (columns T,F for W; rows by S,R in order (F,F), (F,T), (T,F), (T,T)):
    # (F,F): T 0.0, F 1.0
    # (F,T): T 0.8, F 0.2
    # (T,F): T 0.9, F 0.1
    # (T,T): T 0.99, F 0.01
    'WetGrass': {
        #(W,S,R):
        (1,0,0): 0.0,  (0,0,0): 1.0,
        (1,0,1): 0.8,  (0,0,1): 0.2,
        (1,1,0): 0.9,  (0,1,0): 0.1,
        (1,1,1): 0.99, (0,1,1): 0.01,
    }
}

In [10]:
print("P(Rain | WetGrass=1) =", {k: round(v,3) for k,v in query_posterior('Rain', {'WetGrass':1}, cpts).items()})


P(Rain | WetGrass=1) = {0: 0.23, 1: 0.77}


In [ ]:
#You can see that once we 
#change the percentage of the likelihood of rain, 
#the probability of the 
#rain being the reason for the wet grass increases

In [7]:
cpts = {
    'Rain': {0: 0.8, 1: 0.2},  # P(R=F)=0.8, P(R=T)=0.2
    # Sprinkler | Rain  (S, R): probabilities for S state given R
    # From table under "SPRINKLER": when R=F -> [T:0.4, F:0.6]; when R=T -> [T:0.01, F:0.99]
    'Sprinkler': {
        #(S,R): P(S|R)
        (0,0): 0.6,  # P(S=0|R=0)
        (1,0): 0.4,  # P(S=1|R=0)
        (0,1): 0.95, # P(S=0|R=1)
        (1,1): 0.05, # P(S=1|R=1)
    },
    # GrassWet | Sprinkler, Rain  (W, S, R)
    # Table (columns T,F for W; rows by S,R in order (F,F), (F,T), (T,F), (T,T)):
    # (F,F): T 0.0, F 1.0
    # (F,T): T 0.8, F 0.2
    # (T,F): T 0.9, F 0.1
    # (T,T): T 0.99, F 0.01
    'WetGrass': {
        #(W,S,R):
        (1,0,0): 0.0,  (0,0,0): 1.0,
        (1,0,1): 0.8,  (0,0,1): 0.2,
        (1,1,0): 0.9,  (0,1,0): 0.1,
        (1,1,1): 0.95, (0,1,1): 0.05,
    }
}

In [8]:
print("P(Rain | WetGrass=1) =", {k: round(v,3) for k,v in query_posterior('Rain', {'WetGrass':1}, cpts).items()})


P(Rain | WetGrass=1) = {0: 0.641, 1: 0.359}


In [9]:
print("P(Sprinkler | WetGrass=1) =", {k: round(v,3) for k,v in query_posterior('Sprinkler', {'WetGrass':1}, cpts).items()})

P(Sprinkler | WetGrass=1) = {0: 0.338, 1: 0.662}


In [10]:
print("P(Rain | WetGrass=1, Sprinkler=0) =", {k: round(v,3) for k,v in query_posterior('Rain', {'WetGrass':1,'Sprinkler':0}, cpts).items()})

P(Rain | WetGrass=1, Sprinkler=0) = {0: 0.0, 1: 1.0}


In [ ]:
#As expected, a very subtle change to the sprinkler
#being the reason for wet grass given that rain has transpired
#does not alter the posterior outcomes in a large manner

In [12]:
cpts = {
    'Rain': {0: 0.8, 1: 0.2},  # P(R=F)=0.8, P(R=T)=0.2
    # Sprinkler | Rain  (S, R): probabilities for S state given R
    # From table under "SPRINKLER": when R=F -> [T:0.4, F:0.6]; when R=T -> [T:0.01, F:0.99]
    'Sprinkler': {
        #(S,R): P(S|R)
        (0,0): 0.6,  # P(S=0|R=0)
        (1,0): 0.4,  # P(S=1|R=0)
        (0,1): 0.99, # P(S=0|R=1)
        (1,1): 0.01, # P(S=1|R=1)
    },
    # GrassWet | Sprinkler, Rain  (W, S, R)
    # Table (columns T,F for W; rows by S,R in order (F,F), (F,T), (T,F), (T,T)):
    # (F,F): T 0.0, F 1.0
    # (F,T): T 0.8, F 0.2
    # (T,F): T 0.9, F 0.1
    # (T,T): T 0.99, F 0.01
    'WetGrass': {
        #(W,S,R):
        (1,0,0): 0.15,  (0,0,0): 0.85,
        (1,0,1): 0.8,  (0,0,1): 0.2,
        (1,1,0): 0.9,  (0,1,0): 0.1,
        (1,1,1): 0.99, (0,1,1): 0.01,
    }
}

In [13]:
print("P(Rain | WetGrass=1) =", {k: round(v,3) for k,v in query_posterior('Rain', {'WetGrass':1}, cpts).items()})


P(Rain | WetGrass=1) = {0: 0.692, 1: 0.308}


In [ ]:
#Changing this seems to drastically alter the
#percentage indicating that wet grass does not 
#indicate rain

In [14]:
cpts = {
    'FPS Game': {0: 0.5, 1: 0.5},  # P(R=F)=0.8, P(R=T)=0.2
    # Sprinkler | Rain  (S, R): probabilities for S state given R
    # From table under "SPRINKLER": when R=F -> [T:0.4, F:0.6]; when R=T -> [T:0.01, F:0.99]
    'Online Gameplay': {
        #(O,F): P(O|F)
        (0,0): 0.3,  # P(S=0|R=0)
        (1,0): 0.7,  # P(S=1|R=0)
        (0,1): 0.05, # P(S=0|R=1)
        (1,1): 0.95, # P(S=1|R=1)
    },
    # GrassWet | Sprinkler, Rain  (W, S, R)
    # Table (columns T,F for W; rows by S,R in order (F,F), (F,T), (T,F), (T,T)):
    # (F,F): T 0.0, F 1.0
    # (F,T): T 0.8, F 0.2
    # (T,F): T 0.9, F 0.1
    # (T,T): T 0.99, F 0.01
    'Acclaimed Success': {
        #(S,O,F):
        (1,0,0): 0.2,  (0,0,0): 0.8,
        (1,0,1): 0.25,  (0,0,1): 0.75,
        (1,1,0): 0.5,  (0,1,0): 0.5,
        (1,1,1): 0.8, (0,1,1): 0.2,
    }
}

In [16]:
def normalize(pdict):
    z = sum(pdict.values())
    if z == 0:
        raise ValueError("All probabilities are zero; check your CPTs/evidence.")
    return {k: v / z for k, v in pdict.items()}

def all_assignments(vars_list):
    for bits in product([0,1], repeat=len(vars_list)):
        yield dict(zip(vars_list, bits))

def P_joint(assign, cpts):
    r = assign['FPS Game']
    s = assign['Online Gameplay']
    w = assign['Acclaimed Success']
    p = 1.0
    p *= cpts['FPS Game'][r]
    p *= cpts['Online Gameplay'][(s, r)]
    p *= cpts['Acclaimed Success'][(w, s, r)]
    return p

def query_posterior(query_var, evidence, cpts):
    vars_all = ['FPS Game','Online Gameplay','Acclaimed Success']
    hidden = [v for v in vars_all if v != query_var and v not in evidence]
    post_unnorm = {}
    for q_val in [0,1]:
        total = 0.0
        for h_assign in all_assignments(hidden):
            assign = {query_var: q_val, **evidence, **h_assign}
            total += P_joint(assign, cpts)
        post_unnorm[q_val] = total
    return normalize(post_unnorm)


In [17]:
def check_cpts(cpts):
    assert abs(cpts['FPS Game'][0] + cpts['FPS Game'][1] - 1.0) < 1e-9
    for r in [0,1]:
        s0 = cpts['Online Gameplay'][(0,r)]
        s1 = cpts['Online Gameplay'][(1,r)]
        assert abs(s0 + s1 - 1.0) < 1e-9
    for s in [0,1]:
        for r in [0,1]:
            w1 = cpts['Acclaimed Success'][(1,s,r)]
            w0 = cpts['Acclaimed Success'][(0,s,r)]
            assert abs(w0 + w1 - 1.0) < 1e-9
    return "CPTs look valid."

check_cpts(cpts)

'CPTs look valid.'

In [18]:
p_wet = 0.0
for r in [0,1]:
    for s in [0,1]:
        p_wet += cpts['FPS Game'][r] * cpts['Online Gameplay'][(s,r)] * cpts['Acclaimed Success'][(1,s,r)]
print(f"P(Acclaimed Success=1) = {p_wet:.3f}")

P(Acclaimed Success=1) = 0.591


In [19]:
print("P(FPS Game | Acclaimed Success=1) =", {k: round(v,3) for k,v in query_posterior('FPS Game', {'Acclaimed Success':1}, cpts).items()})


P(FPS Game | Acclaimed Success=1) = {0: 0.347, 1: 0.653}


In [20]:
print("P(Online Gameplay | Acclaimed Success=1) =", {k: round(v,3) for k,v in query_posterior('Online Gameplay', {'Acclaimed Success':1}, cpts).items()})

P(Online Gameplay | Acclaimed Success=1) = {0: 0.061, 1: 0.939}


In [22]:
print("P(FPS Game | Acclaimed Success=1, Online Gameplay=0) =", {k: round(v,3) for k,v in query_posterior('FPS Game', {'Acclaimed Success':1,'Online Gameplay':0}, cpts).items()})

P(FPS Game | Acclaimed Success=1, Online Gameplay=0) = {0: 0.828, 1: 0.172}


In [23]:
print("P(FPS Game | Acclaimed Success=1, Online Gameplay=1) =", {k: round(v,3) for k,v in query_posterior('FPS Game', {'Acclaimed Success':1,'Online Gameplay':1}, cpts).items()})

P(FPS Game | Acclaimed Success=1, Online Gameplay=1) = {0: 0.315, 1: 0.685}


In [ ]:
#DISCLAIMER: I did not research actual numbers in the gaming industry of
#which games were successful and which ones had the components added for this test.
#All numbers added as fictional and should not be taken as truth

#My fictional story revolved around the computer and console gaming industry and focused on
#success of games given their components: Online gameplay and First Person Shooter (as this
#is the most popular genre in the market)


## (Optional) Build a tiny custom BN (≤ 4 binary variables)


In [12]:

def P_joint_generic(assign, vars_topo, parents, cpts):
    p = 1.0
    for v in vars_topo:
        pa = parents[v]
        if len(pa) == 0:
            p *= cpts[v][assign[v]]
        else:
            key = (assign[v],) + tuple(assign[u] for u in pa)
            p *= cpts[v][key]
    return p

def query_generic(query_var, evidence, vars_topo, parents, cpts):
    from itertools import product
    hidden = [v for v in vars_topo if v != query_var and v not in evidence]
    post = {}
    for q in [0,1]:
        total = 0.0
        for bits in product([0,1], repeat=len(hidden)):
            h = dict(zip(hidden, bits))
            a = {query_var:q, **evidence, **h}
            total += P_joint_generic(a, vars_topo, parents, cpts)
        post[q] = total
    # normalize
    z = sum(post.values())
    return {k: v / z for k, v in post.items()}

# Example reuse
vars_topo = ['Rain','Sprinkler','WetGrass']
parents = {'Rain':[], 'Sprinkler':['Rain'], 'WetGrass':['Sprinkler','Rain']}
cpts_generic = {
    'Rain': {0:0.7, 1:0.3},
    'Sprinkler': {(0,0):0.2,(1,0):0.8,(0,1):0.7,(1,1):0.3},
    'WetGrass':{
        (1,1,1):0.99,(0,1,1):0.01,(1,1,0):0.90,(0,1,0):0.10,
        (1,0,1):0.80,(0,0,1):0.20,(1,0,0):0.05,(0,0,0):0.95,
    }
}
query_generic('Rain', {'WetGrass':1}, vars_topo, parents, cpts_generic)


{0: 0.6652779585991407, 1: 0.33472204140085926}